In [1]:
from operator import is_
import os
import sys
sys.path.append('../.')
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
import numpy as np

import clip

from Metrics import base_kmeans_model_evaluation, kmeans_with_init, cosine_kmeans_with_init
from networks import CustomCLIP, load_clip_to_cpu
from lr_scheduler import ConstantWarmupScheduler

import argparse

In [2]:
clip_backbone = "ViT-L/14"
print(clip.available_models())
device = torch.device('cuda') if torch.cuda.is_available() else "cpu"
print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

backbone_name = clip_backbone
clip_model, preprocess = load_clip_to_cpu(backbone_name)


['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']
Device: cuda
Current cuda device: 0
Count of using GPUs: 4


In [23]:
batch_size =100
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=preprocess)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False)

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                       download=True, transform=preprocess)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                         shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
model = CustomCLIP(clip_model, len(testset.classes))

Initializing class-specific contexts
Initial context: "X X X X X X X X X X X X X X X X"
Number of context words (tokens): 16


In [43]:
from torch.autograd import Variable
from sklearn.cluster import KMeans
from Metrics import nmi, acc
def base_kmeans_model_evaluation(model, data_loader, num_cluster):
    model.eval()
    with torch.no_grad():
        dummy = torch.rand(1,3,224,224).cuda()
        feature_dim = model.image_encoder(dummy).shape[-1]
        print(feature_dim)
        dataset_size = len(data_loader.dataset)
        datas = np.zeros([dataset_size, feature_dim])
        label_true = np.zeros(dataset_size)
        ii = 0
        for x, target in data_loader:
            b = x.shape[0]
            x = Variable(x).cuda()
            
            image_representation = model.image_encoder(x)
#             image_representation = model.encode_image(x.type(model.dtype))
            image_representation = image_representation / image_representation.norm(dim=-1, keepdim=True)
            u = image_representation.cpu()
            datas[ii * data_loader.batch_size:(ii + 1) *
                  data_loader.batch_size, :] = u.data.numpy()
            label_true[ii * data_loader.batch_size:(ii + 1)
                       * data_loader.batch_size] = target.numpy()
            ii = ii + 1
            print(ii)
        kmeans = KMeans(n_clusters=num_cluster, random_state=0).fit(datas)
        label_pred = kmeans.labels_
        centroids = kmeans.cluster_centers_
        ACC = acc(label_true, label_pred, num_cluster)
        NMI = nmi(label_true, label_pred)
        print('image_ACC', ACC)
        print('image_NMI', NMI)
        return centroids, label_pred, ACC, NMI

In [45]:
num_classes = 10 
with torch.no_grad():
    centroids, test_label, acc, nmi= base_kmeans_model_evaluation(
        model.to(device), trainloader, num_classes)

768
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276


TypeError: 'numpy.float64' object is not callable

In [14]:

ACC = acc(testset.targets, test_label, num_classes)
NMI = nmi(testset.targets, test_label)
print('image_ACC', ACC)
print('image_NMI', NMI)

NameError: name 'test_label' is not defined

In [ ]:
num_classes = 10 
with torch.no_grad():
    train_centroids, test_label, acc, nmi= base_kmeans_model_evaluation(
        clip_model.to(device), trainloader, num_classes)

In [164]:
X = centroids
X_square = X**2
X_square_sum = np.sum(X_square,axis=1)
X_train_square = X**2
X_square_train_sum = np.sum(X_train_square,axis=1)
XY = X@X.T
# dists_original = np.sqrt(X_square_sum.reshape(-1,1)+X_square_train_sum.reshape(1,-1)-2*XY)
dists = np.sqrt(X_square_sum.reshape(-1,1)+X_square_train_sum.reshape(1,-1)-2*XY+1e-12)

In [165]:
np.argmin(dists,axis=-1)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [166]:
dists.argsort(axis=-1)[:,1]

array([9, 0, 7, 5, 8, 9, 9, 2, 4, 5])

In [167]:
k = dists[np.arange(10),dists.argsort(axis=-1)[:,1]]

In [100]:
c

array([[-0.0110123 ,  0.03005093, -0.01091487, ...,  0.00862239,
         0.00132192, -0.0172122 ],
       [ 0.00724263,  0.0220054 ,  0.00479171, ..., -0.00250016,
        -0.00948331, -0.02184404],
       [ 0.00571699,  0.03417663, -0.00728394, ...,  0.00621605,
        -0.00533025, -0.02589159],
       ...,
       [ 0.00849806,  0.04163024,  0.00323967, ...,  0.01474007,
         0.01407203, -0.02373944],
       [-0.01054857,  0.02590865, -0.01196582, ...,  0.03023189,
        -0.00372186, -0.02541586],
       [-0.01246983,  0.03131478,  0.00270114, ...,  0.00727812,
        -0.00256862, -0.01381275]])

In [6]:
classes_list = ['a photo of a '+ c for c in testset.classes]
classes_list
with torch.no_grad():
    classes_list = clip.tokenize(classes_list).to(device)
    centroid_candidate_text = clip_model.to(device).encode_text(classes_list)
    centroid_candidate_text = centroid_candidate_text / centroid_candidate_text.norm(dim=-1, keepdim=True)
for i, (x, target) in enumerate(trainloader):
    x = x.to(device)
    if i ==0:
        break
image_features = model.image_encoder(x.type(clip_model.dtype))
image_features = image_features / image_features.norm(dim=-1, keepdim=True)
sim = image_features@centroid_candidate_text.t()

In [7]:
x = (image_features**2).cpu().detach().numpy()
np.square(np.sum(x,axis=-1))

np.random.rand(1,10)

array([[0.92066314, 0.54009613, 0.16150918, 0.44922276, 0.96773238,
        0.69377026, 0.31059715, 0.82568712, 0.7169295 , 0.8720446 ]])

In [8]:
X = image_features.detach().cpu().numpy()
X_square = X**2
X_square_sum = np.sum(X_square,axis=1)
X_train = train_centroids
X_train_square = X_train**2
X_square_train_sum = np.sum(X_train_square,axis=1)
XY = X@X_train.T
# dists_original = np.sqrt(X_square_sum.reshape(-1,1)+X_square_train_sum.reshape(1,-1)-2*XY)
dists = np.sqrt(X_square_sum.reshape(-1,1)+X_square_train_sum.reshape(1,-1)-2*XY+1e-12)

NameError: name 'train_centroids' is not defined

In [9]:
centroid_candidate_text

tensor([[-0.0237, -0.0017,  0.0569,  ...,  0.0322,  0.0125, -0.0069],
        [ 0.0119, -0.0088,  0.0164,  ...,  0.0056,  0.0435, -0.0258],
        [-0.0066,  0.0147,  0.0375,  ...,  0.0239,  0.0115,  0.0017],
        ...,
        [ 0.0291,  0.0064,  0.0308,  ...,  0.0106, -0.0188, -0.0157],
        [-0.0145, -0.0089,  0.0215,  ...,  0.0299,  0.0108, -0.0028],
        [-0.0124, -0.0029,  0.0378,  ..., -0.0095,  0.0779,  0.0179]],
       device='cuda:0')

In [31]:
clip_backbone = "ViT-L/14"
print(clip.available_models())
device = torch.device('cuda') if torch.cuda.is_available() else "cpu"
print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

backbone_name = clip_backbone
clip_model, preprocess = load_clip_to_cpu(backbone_name)

model = CustomCLIP(clip_model, len(testset.classes))
model.to(device)

['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14']
Device: cuda
Current cuda device: 0
Count of using GPUs: 4
Initializing class-specific contexts
Initial context: "X X X X X X X X X X X X X X X X"
Number of context words (tokens): 16


CustomCLIP(
  (prompt_learner): PromptLearner()
  (image_encoder): VisionTransformer(
    (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
          )
          (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear

In [26]:
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=3e-5,
    weight_decay=5e-4,
    betas=(0.9, 0.999),
)

In [27]:
for i in range(100):
    prompts = model.prompt_learner().cuda()
    tokenized_prompts = model.tokenized_prompts
    text_features = model.text_encoder(prompts, tokenized_prompts)
    loss = torch.mean((text_features-centroid_candidate_text)**2)
    print(loss)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

tensor(0.2303, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1389, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1090, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0860, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0640, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0470, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0337, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0245, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0177, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0132, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0098, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0077, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0063, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0053, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0049, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0047, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0045, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0043, device='cuda:0',

In [32]:
prompts = model.prompt_learner().cuda()
tokenized_prompts = model.tokenized_prompts
text_features = model.text_encoder(prompts, tokenized_prompts)
with torch.no_grad():
    knn = kmeans_with_init
    new_label, acc, nmi = knn(
        model, testloader, num_classes, text_features)

image_ACC 0.3578
image_NMI 0.40012911595880146


In [38]:
lr_list = [1e-3,1e-4,1e-5,1e-6]
lr_list.reverse()
criterion = nn.MSELoss()
for lr in lr_list:
    clip_backbone = "ViT-L/14"
    print(clip.available_models())
    device = torch.device('cuda') if torch.cuda.is_available() else "cpu"
    print('Device:', device)
    print('Current cuda device:', torch.cuda.current_device())
    print('Count of using GPUs:', torch.cuda.device_count())

    backbone_name = clip_backbone
    clip_model, preprocess = load_clip_to_cpu(backbone_name)

    model = CustomCLIP(clip_model, len(testset.classes))
    model.to(device)
    
    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=lr,
        weight_decay=5e-4,
        betas=(0.9, 0.999),
    )
    for i in range(1000):
        prompts = model.prompt_learner().cuda()
        tokenized_prompts = model.tokenized_prompts
        text_features = model.text_encoder(prompts, tokenized_prompts)
        loss = criterion(text_features,centroid_candidate_text)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    model.eval()
    with torch.no_grad():
        knn = kmeans_with_init
        new_label, acc, nmi = knn(
            model, testloader, num_classes, text_features)
    

['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14']
Device: cuda
Current cuda device: 0
Count of using GPUs: 4
Initializing class-specific contexts
Initial context: "X X X X X X X X X X X X X X X X"
Number of context words (tokens): 16
image_ACC 0.3428
image_NMI 0.49615764253114203
['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14']
Device: cuda
Current cuda device: 0
Count of using GPUs: 4
Initializing class-specific contexts
Initial context: "X X X X X X X X X X X X X X X X"
Number of context words (tokens): 16
image_ACC 0.9461
image_NMI 0.8897415806449404
['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14']
Device: cuda
Current cuda device: 0
Count of using GPUs: 4
Initializing class-specific contexts
Initial context: "X X X X X X X X X X X X X X X X"
Number of context words (tokens): 16
image_ACC 0.9596
image_NMI 0.9083590283793902
['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', '

In [46]:
lr_list = [1e-3,1e-4,1e-5,1e-6]
lr_list.reverse()
criterion = nn.MSELoss()
for lr in lr_list:
    clip_backbone = "ViT-L/14"
    print(clip.available_models())
    device = torch.device('cuda') if torch.cuda.is_available() else "cpu"
    print('Device:', device)
    print('Current cuda device:', torch.cuda.current_device())
    print('Count of using GPUs:', torch.cuda.device_count())

    backbone_name = clip_backbone
    clip_model, preprocess = load_clip_to_cpu(backbone_name)

    model = CustomCLIP(clip_model, len(testset.classes))
    model.to(device)
    
    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=lr,
        weight_decay=5e-4,
        betas=(0.9, 0.999),
    )
    for i in range(1000):
        prompts = model.prompt_learner().cuda()
        tokenized_prompts = model.tokenized_prompts
        text_features = model.text_encoder(prompts, tokenized_prompts)
        loss = criterion(text_features,centroid_candidate_text)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    model.eval()
    with torch.no_grad():
        knn = kmeans_with_init
        new_label, acc, nmi = knn(
            model, testloader, num_classes, text_features)
    

['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14']
Device: cuda
Current cuda device: 0
Count of using GPUs: 4
Initializing class-specific contexts
Initial context: "X X X X X X X X X X X X X X X X"
Number of context words (tokens): 16
image_ACC 0.9139
image_NMI 0.8341012580910323
['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14']
Device: cuda
Current cuda device: 0
Count of using GPUs: 4
Initializing class-specific contexts
Initial context: "X X X X X X X X X X X X X X X X"
Number of context words (tokens): 16
image_ACC 0.9583
image_NMI 0.9044289613370075
['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14']
Device: cuda
Current cuda device: 0
Count of using GPUs: 4
Initializing class-specific contexts
Initial context: "X X X X X X X X X X X X X X X X"
Number of context words (tokens): 16
image_ACC 0.9518
image_NMI 0.8942430310189622
['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'V

In [48]:
lr_list = [3e-5]
lr_list.reverse()
criterion = nn.MSELoss()
for lr in lr_list:
    clip_backbone = "ViT-L/14"
    print(clip.available_models())
    device = torch.device('cuda') if torch.cuda.is_available() else "cpu"
    print('Device:', device)
    print('Current cuda device:', torch.cuda.current_device())
    print('Count of using GPUs:', torch.cuda.device_count())

    backbone_name = clip_backbone
    clip_model, preprocess = load_clip_to_cpu(backbone_name)

    model = CustomCLIP(clip_model, len(testset.classes))
    model.to(device)
    
    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=lr,
        weight_decay=5e-4,
        betas=(0.9, 0.999),
    )
    for i in range(1000):
        prompts = model.prompt_learner().cuda()
        tokenized_prompts = model.tokenized_prompts
        text_features = model.text_encoder(prompts, tokenized_prompts)
        loss = criterion(text_features,centroid_candidate_text)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    model.eval()
    with torch.no_grad():
        knn = kmeans_with_init
        new_label, acc, nmi = knn(
            model, testloader, num_classes, text_features)
    

['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14']
Device: cuda
Current cuda device: 0
Count of using GPUs: 4
Initializing class-specific contexts
Initial context: "X X X X X X X X X X X X X X X X"
Number of context words (tokens): 16
image_ACC 0.9583
image_NMI 0.9044871435275373
